# Ungraded Lab: Huber Loss hyperparameter and class

In this lab, we'll extend our previous Huber loss function and show how you can include hyperparameters in defining loss functions. We'll also look at how to implement a custom loss as an object by inheriting the [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/Loss) class.

In [1]:
import tensorflow as tf
import numpy as np

2025-01-24 13:17:12.819348: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 13:17:12.828923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737724632.840372  715296 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737724632.843413  715296 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 13:17:12.854793: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Dataset

As before, this model will be trained on the `xs` and `ys` below where the relationship is $y = 2x-1$. Thus, later, when we test for `x=10`, whichever version of the model gets the closest answer to `19` will be deemed more accurate.

In [2]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Custom loss with hyperparameter

The `loss` argument in `model.compile()` only accepts functions that accepts two parameters: the ground truth (`y_true`) and the model predictions (`y_pred`). If we want to include a hyperparameter that we can tune, then we can define a wrapper function that accepts this hyperparameter.

In [3]:
def my_huber_loss_with_threshold(threshold: float):

    def my_huber_loss(y_true: float, y_pred: float):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

    return my_huber_loss

We can now specify the `loss` as the wrapper function above. Notice that we can now set the `threshold` value. Try varying this value and see the results you get.

In [4]:
model = tf.keras.models.Sequential(
    layers=[
        tf.keras.layers.Dense(units=1, input_shape=[1])
    ]
)

/home/mbarbaric/dev/ai/tensorflow_advanced_techniques_specialization/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1737725302.277864  715296 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5566 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1.2))

In [6]:
model.fit(xs, ys, epochs=500, verbose=0)

I0000 00:00:1737725334.826310  717553 service.cc:148] XLA service 0x7f5a380050c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737725334.826378  717553 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-01-24 13:28:54.836956: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737725334.858572  717553 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1737725334.952416  717553 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [7]:
print(model.predict(np.array([10.0], dtype=float)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
[[18.7822]]


## Implement Custom Loss as a Class

We can also implement our custom loss as a class. It inherits from the Keras Loss class and the syntax and required methods are shown below.

In [8]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1):
        super().__init__()
        self.threshold = threshold

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [9]:
model = tf.keras.models.Sequential(
    layers=[
        tf.keras.layers.Dense(units=1, input_shape=[1])
    ]
)

/home/mbarbaric/dev/ai/tensorflow_advanced_techniques_specialization/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1.2))

In [11]:
model.fit(xs, ys, epochs=500, verbose=0)

In [12]:
print(model.predict(np.array([10.0], dtype=float)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
[[18.576786]]
